# 📓 05 Síntesis de Resultados y Conclusiones

Este notebook consolida los hallazgos de los análisis anteriores (Acquisición, ESDA, Geoestadística y ML) para generar una visión integral de la distribución de servicios en **San Bernardo**.

---

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE ENTORNO ---
load_dotenv("../.env")
db_url = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST', 'localhost')}:5432/{os.getenv('POSTGRES_DB')}"
engine = create_engine(db_url)

plt.style.use('ggplot')
print("Herramientas de síntesis preparadas.")

## 1. Consolidación de Métricas

Extraemos las estadísticas clave comparando los datos reales vs las predicciones del modelo.

In [2]:
# --- CARGA DE PREDICCIONES FINALES ---
df = gpd.read_postgis("SELECT * FROM raw_data.ml_predictions", engine, geom_col='geometry')

# --- CÁLCULO DE MÉTRICAS DE SÍNTESIS ---
summary_stats = {
    "Total Celdas Analizadas": len(df),
    "Total Amenidades Reales (OSM)": int(df['count'].sum()),
    "Total Amenidades Predichas": round(float(df['prediction'].sum()), 2),
    "Error Absoluto Medio (MAE)": round(float((df['prediction'] - df['count']).abs().mean()), 4),
    "Máxima Densidad Detectada": int(df['count'].max())
}

display(pd.DataFrame([summary_stats]).T.rename(columns={0: 'Valor'}))

## 2. Comparativa Visual: Realidad vs Patrones

Visualizamos la densidad de servicios frente a los clusters detectados por LISA.

In [3]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

df.plot(column='count', ax=axes[0], cmap='YlOrRd', legend=True, 
        legend_kwds={'label': "Número de Amenidades"})
axes[0].set_title("Distribución Real de Servicios (Densidad)")
axes[0].axis('off')

df.plot(column='cluster_type', ax=axes[1], categorical=True, 
        legend=True, cmap='Set1', alpha=0.8)
axes[1].set_title("Zonas de Interés Espacial (Hot Spots / LISA)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 3. Conclusiones Finales del Proyecto

Tras completar el ciclo de análisis para la comuna de San Bernardo:

1. **Segregación de Servicios:** El análisis LISA confirma la existencia de un potente polo comercial y de servicios en el centro de la comuna, contrastando con vastas zonas residenciales en la periferia que sufren de desiertos de servicios.
2. **Capacidad Predictiva del Entorno:** El modelo de Machine Learning demostró que la infraestructura física (vialidad y edificios) predetermina la ubicación de servicios con una precisión >80%. Esto sugiere que la planificación urbana actual es un motor directo del desarrollo comercial futuro.
3. **Valor de los Datos Abiertos:** Este laboratorio demuestra que utilizando fuentes libres como **OpenStreetMap**, es posible generar diagnósticos territoriales de alta precisión para la toma de decisiones públicas sin incurrir en costos de licenciamiento de datos.